In [1]:
import keras

from keras.datasets import cifar10

from keras.models import Model, Sequential

from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation

from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.layers import Concatenate

from keras.optimizers import Adam, SGD

from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory

# backend

import tensorflow as tf

from keras import backend as k

# Don't pre-allocate memory; allocate as-needed

config = tf.ConfigProto()

config.gpu_options.allow_growth = True

# Create a session with the above options specified.

k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
# Hyperparameters

batch_size = 64

num_classes = 10

epochs = 250

l = 40

num_filter = 12

compression = 0.5

dropout_rate = 0.2

In [4]:
# Load CIFAR10 Data

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]


# convert to one hot encoing 

y_train = keras.utils.to_categorical(y_train, num_classes)

y_test = keras.utils.to_categorical(y_test, num_classes)



x_train=x_train.astype('float32')

x_test = x_test.astype('float32')

x_train/=255

x_test/=255

In [5]:
# Dense Block

def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):

    global compression

    temp = input

    for _ in range(l):

        BatchNorm = BatchNormalization()(temp)

        relu = Activation('relu')(BatchNorm)

        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)

        if dropout_rate>0:

          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)

        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])

        

        temp = concat

        

    return temp



def add_transition(input, num_filter = 12, dropout_rate = 0.2):

    global compression

    BatchNorm = BatchNormalization()(input)

    relu = Activation('relu')(BatchNorm)

    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)

    if dropout_rate>0:

      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)

    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)

    

    return avg


def output_layer(input):

    global compression

    BatchNorm = BatchNormalization()(input)

    relu = Activation('relu')(BatchNorm)

    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)

    flat = Flatten()(AvgPooling)

    output = Dense(num_classes, activation='softmax')(flat)

    

    return output

In [6]:
input = Input(shape=(img_height, img_width, channel,))

First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)

First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)

Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)

Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)

output = output_layer(Third_Transition)

In [7]:
model = Model(inputs=[input], outputs=[output])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [8]:
datagen = ImageDataGenerator(horizontal_flip=True,
                             rotation_range=15,
                             width_shift_range=0.15,
                             height_shift_range=0.15,
                             fill_mode='constant')

datagen.fit(x_train)

In [9]:
from keras.callbacks import LearningRateScheduler

def lr_decay_callback():

    def step_decay(epoch):

        if 0<epoch<=125:

            learn_rate=0.1

            return learn_rate

        elif 125<epoch<=188:

            learn_rate=0.01

            return learn_rate

        elif 188<epoch<=210:

            learn_rate=0.001

            return learn_rate

        elif 210<epoch<=225:

            learn_rate=0.0001

            return learn_rate

        else:

            learn_rate=0.00001

            return learn_rate

    return LearningRateScheduler(step_decay)

In [10]:
learning_rate = 0.1

#decay_rate = learning_rate / epochs

momentum = 0.9

opt = SGD(lr=learning_rate, momentum=momentum, nesterov=True)

# determine Loss function and Optimizer

model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

In [11]:
from keras.callbacks import ModelCheckpoint

augmented_checkpoint = ModelCheckpoint('a_1.h5',  # model filename

                             monitor='val_loss', # quantity to monitor

                             verbose=0, # verbosity - 0 or 1

                             save_best_only= True, # The latest best model will not be overwritten

                             mode='auto') # The decision to overwrite model is made 

                                          # automatically depending on the quantity to monitor

    

In [12]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size, shuffle=True),

                                                  steps_per_epoch = len(x_train) / 32, # number of samples per gradient update

                                                  epochs = epochs, # number of iterations

                                                  validation_data= (x_test, y_test),

                                                  callbacks=[lr_decay_callback(),augmented_checkpoint],

                                                  verbose=1)

Epoch 1/250
1563/1562 [==============================] - 347s 222ms/step - loss: 2.4209 - acc: 0.0995 - val_loss: 2.3161 - val_acc: 0.1052
Epoch 2/250
1563/1562 [==============================] - 314s 201ms/step - loss: 1.6327 - acc: 0.3937 - val_loss: 2.2563 - val_acc: 0.4142
Epoch 3/250
1563/1562 [==============================] - 318s 203ms/step - loss: 1.2389 - acc: 0.5537 - val_loss: 3.1898 - val_acc: 0.4204
Epoch 4/250
1563/1562 [==============================] - 316s 202ms/step - loss: 1.0691 - acc: 0.6207 - val_loss: 2.2134 - val_acc: 0.5057
Epoch 5/250
1563/1562 [==============================] - 318s 203ms/step - loss: 0.9536 - acc: 0.6633 - val_loss: 1.3120 - val_acc: 0.6609
Epoch 6/250
1563/1562 [==============================] - 318s 204ms/step - loss: 0.8771 - acc: 0.6930 - val_loss: 1.2217 - val_acc: 0.6893
Epoch 7/250
1563/1562 [==============================] - 321s 205ms/step - loss: 0.8155 - acc: 0.7156 - val_loss: 1.3288 - val_acc: 0.6569
Epoch 8/250
1563/1562 [====

1563/1562 [==============================] - 315s 201ms/step - loss: 0.2418 - acc: 0.9173 - val_loss: 0.4069 - val_acc: 0.8958
Epoch 118/250
1563/1562 [==============================] - 316s 202ms/step - loss: 0.2387 - acc: 0.9174 - val_loss: 0.3990 - val_acc: 0.8940
Epoch 119/250
1563/1562 [==============================] - 318s 203ms/step - loss: 0.2376 - acc: 0.9176 - val_loss: 0.4325 - val_acc: 0.8911
Epoch 120/250
1563/1562 [==============================] - 317s 203ms/step - loss: 0.2330 - acc: 0.9201 - val_loss: 0.3996 - val_acc: 0.9013
Epoch 121/250
1563/1562 [==============================] - 316s 202ms/step - loss: 0.2371 - acc: 0.9182 - val_loss: 0.4502 - val_acc: 0.8884
Epoch 122/250
1563/1562 [==============================] - 317s 203ms/step - loss: 0.2355 - acc: 0.9185 - val_loss: 0.4653 - val_acc: 0.8805
Epoch 123/250
1563/1562 [==============================] - 318s 204ms/step - loss: 0.2327 - acc: 0.9201 - val_loss: 0.3551 - val_acc: 0.9030
Epoch 124/250
1563/1562 [==

1563/1562 [==============================] - 320s 205ms/step - loss: 0.1403 - acc: 0.9511 - val_loss: 0.3774 - val_acc: 0.9167
Epoch 234/250
1563/1562 [==============================] - 318s 203ms/step - loss: 0.1393 - acc: 0.9513 - val_loss: 0.3779 - val_acc: 0.9163
Epoch 235/250
1563/1562 [==============================] - 320s 205ms/step - loss: 0.1414 - acc: 0.9508 - val_loss: 0.3781 - val_acc: 0.9162
Epoch 236/250
1563/1562 [==============================] - 319s 204ms/step - loss: 0.1392 - acc: 0.9516 - val_loss: 0.3776 - val_acc: 0.9162
Epoch 237/250
1563/1562 [==============================] - 318s 204ms/step - loss: 0.1388 - acc: 0.9513 - val_loss: 0.3761 - val_acc: 0.9161
Epoch 238/250
1563/1562 [==============================] - 318s 203ms/step - loss: 0.1395 - acc: 0.9512 - val_loss: 0.3766 - val_acc: 0.9165
Epoch 239/250
1563/1562 [==============================] - 316s 202ms/step - loss: 0.1398 - acc: 0.9511 - val_loss: 0.3774 - val_acc: 0.9168
Epoch 240/250
1563/1562 [==